In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


<b>Pregnancies</b> — количество беременностей.

<b>Glucose</b> — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

<b>BloodPressure</b> — диастолическое артериальное давление (мм рт. ст.).

<b>SkinThickness</b> — толщина кожной складки трицепса (мм).

<b>Insulin</b> — двухчасовой сывороточный инсулин (ме Ед/мл).

<b>BMI</b> — индекс массы тела (вес в кг / (рост в м)**2)

<b>DiabetesPedigreeFunction</b> — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

<b>Age</b> — возраст.

<b>Outcome</b> — наличие диабета (0 — нет, 1 — да).

In [2]:
diabetes.shape

(778, 10)

In [3]:
#8.1
diabetes = diabetes.drop_duplicates(subset=list(diabetes.columns))
print(f'Результирующее число записей: {diabetes.shape[0]}')

Результирующее число записей: 768


In [4]:
#8.2
low_information_cols = [] 

for col in diabetes.columns:
    top_freq = diabetes[col].value_counts(normalize=True).max()
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes = diabetes.drop(low_information_cols, axis=1)

Gender: 100.0% одинаковых значений


In [5]:
#8.3
diabetes['Glucose'] = diabetes['Glucose'].replace(0, np.nan)
diabetes['BloodPressure'] = diabetes['BloodPressure'].replace(0, np.nan)
diabetes['SkinThickness'] = diabetes['SkinThickness'].replace(0, np.nan)
diabetes['Insulin'] = diabetes['Insulin'].replace(0, np.nan)
diabetes['BMI'] = diabetes['BMI'].replace(0, np.nan)

round(diabetes.isnull().mean(),2)

Pregnancies                 0.00
Glucose                     0.01
BloodPressure               0.05
SkinThickness               0.30
Insulin                     0.49
BMI                         0.01
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [6]:
#8.4

thresh = diabetes.shape[0]*0.7
diabetes = diabetes.dropna(thresh=thresh, axis=1)
diabetes.shape

(768, 8)

In [7]:
#8.5
m = diabetes.shape[1]
diabetes = diabetes.dropna(thresh=m-2, axis=0)
print(diabetes.shape[0])

761


In [8]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 761 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               761 non-null    int64  
 1   Glucose                   756 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  761 non-null    float64
 6   Age                       761 non-null    int64  
 7   Outcome                   761 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 53.5 KB


In [9]:
#8.6
values = {
    'Pregnancies': diabetes['Pregnancies'].median(),
    'Glucose': diabetes['Glucose'].median(),
    'BloodPressure': diabetes['BloodPressure'].median(),
    'SkinThickness': diabetes['SkinThickness'].median(),
    'BMI': diabetes['BMI'].median(),
    'DiabetesPedigreeFunction': diabetes['DiabetesPedigreeFunction'].median(),
    'Age': diabetes['Age'].median(),
    'Outcome': diabetes['Outcome'].median()
}
diabetes = diabetes.fillna(values)
diabetes['SkinThickness'].mean()

29.109067017082786

In [10]:
#8.7
def outliers_iqr(data, feature):
    x = data[feature]
    # вычислить 25-ый и 75-ый квантили (первый и третий квартили): 
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    # вычислить межквартильное расстояние:
    iqr = quartile_3 - quartile_1
    # вычислить верхнюю и нижнюю границы Тьюки:
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    # найти наблюдения, которые выходят за пределы границ:
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [11]:
#8.8
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(diabetes, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


In [12]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    # вычислить 25-ый и 75-ый квантили (первый и третий квартили): 
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    # вычислить межквартильное расстояние:
    iqr = quartile_3 - quartile_1
    # вычислить верхнюю и нижнюю границы Тьюки:
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    # найти наблюдения, которые выходят за пределы границ:
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [15]:
#8.9
outliers, cleaned = outliers_iqr(diabetes, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 29
Результирующее число записей: 732


In [17]:
outliers1, cleaned1 = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers1.shape[0]}')
print(f'Результирующее число записей: {cleaned1.shape[0]}')

Число выбросов по методу z-отклонения: 0
Результирующее число записей: 761
